# Proteus Well-log data

This notebook demonstrates how to work with Proteus well-log data using GeoSuite.

## Overview

This notebook will show you how to:

1. Load well-log data
2. Process and analyze the data
3. Visualize well logs

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs

print("GeoSuite imported successfully!")

In [ ]:
# Load demo well log data
df = load_demo_well_logs()

print(f"Loaded {len(df):,} data points")
print(f"\nAvailable columns: {df.columns.tolist()}")
df.head()

In [ ]:
# Display basic statistics
print("Well Log Statistics:")
print(df.describe())

## 3. Visualize Well Logs

Create visualizations of the well log data.

In [ ]:
# Create strip chart visualization
try:
    
    depth_col = 'depth_m' if 'depth_m' in df.columns else 'DEPTH'
    fig = create_strip_chart(
        df=df,
        depth_col=depth_col,
        title='Proteus Well Log Data'
    )
    plt.show()
    
except ImportError:
    # Simple visualization if plotting module not available
    depth_col = 'depth_m' if 'depth_m' in df.columns else df.columns[0]
    
    fig, axes = plt.subplots(1, min(3, len(df.columns)-1), figsize=(12, 8), sharey=True)
    if not isinstance(axes, np.ndarray):
        axes = [axes]
    
    depth = df[depth_col].values
    
    numeric_cols = [col for col in df.columns if col != depth_col and pd.api.types.is_numeric_dtype(df[col])]
    
    for idx, col in enumerate(numeric_cols[:3]):
        if idx < len(axes):
            axes[idx].plot(df[col], depth, linewidth=0.5)
            axes[idx].set_xlabel(col)
            axes[idx].set_title(col)
            axes[idx].invert_yaxis()
            axes[idx].grid(True, alpha=0.3)
    
    axes[0].set_ylabel('Depth')
    plt.tight_layout()
    plt.show()

## 4. Basic Petrophysical Analysis

Perform basic petrophysical calculations on the well log data.

In [ ]:
# Example: Calculate basic petrophysical properties
try:
    
    # Try to calculate porosity from density if available
    density_cols = [col for col in df.columns if 'RHOB' in col.upper() or 'DENSITY' in col.upper()]
    
    if density_cols:
        density_col = density_cols[0]
        print(f"Calculating porosity from {density_col}")
        
        # Calculate porosity (assuming sandstone matrix density)
        porosity = calculate_porosity_from_density(
            df[density_col].values,
            rho_matrix=2.65,  # g/cc for sandstone
            rho_fluid=1.0     # g/cc for fresh water
        )
        
        df['Porosity_Density'] = porosity
        print(f"Porosity range: {porosity.min():.3f} to {porosity.max():.3f}")
        print(f"Mean porosity: {porosity.mean():.3f}")
    else:
        print("No density column found for porosity calculation")
        
except Exception as e:
    print(f"Porosity calculation not available: {e}")

print("\nData processing complete!")

## 5. Summary

This notebook demonstrated:

- Loading well log data (demo data as example)
- Displaying statistics and data exploration
- Creating professional visualizations with `create_strip_chart()`
- Calculating basic petrophysical properties

### Next Steps

- Load your own Proteus or other well log data formats
- Use `geosuite.petro.pickett_plot()` for reservoir analysis
- Use `geosuite.petro.buckles_plot()` for reservoir quality assessment
- Use `geosuite.ml` for facies classification
- Export processed results to your preferred format